# Circuit simulation examples

In [1]:
import obi_one as obi

In [2]:
# circuit_path_prefix = "/Users/james/Documents/obi/additional_data/"
circuit_path_prefix = "/Users/pokorny/Data/Circuits/"

### Loading two circuits

In [3]:
circuit = obi.Circuit(name="ToyCircuit-S1-6k", path=circuit_path_prefix + "ToyCircuit-S1-6k/circuit_config.json")
print(f"Circuit '{circuit}' with {circuit.sonata_circuit.nodes.size} neurons and {circuit.sonata_circuit.edges.size} synapses")

Circuit 'ToyCircuit-S1-6k' with 5924 neurons and 568717 synapses


In [4]:
# circuit2 = obi.Circuit(name="ToyCircuit-S1-6k-duplicate", path="/Users/pokorny/Data/Circuits/ToyCircuit-S1-6k/circuit_config.json")
circuit2 = obi.Circuit(name="nbS1-HEX0-beta", path=circuit_path_prefix + "ToyCircuit-S1-6k/circuit_config.json")
print(f"Circuit '{circuit2}' with {circuit2.sonata_circuit.nodes.size} neurons and {circuit2.sonata_circuit.edges.size} synapses")

Circuit 'nbS1-HEX0-beta' with 5924 neurons and 568717 synapses


### Defining the circuit simulation

In [5]:
sim_neuron_set = obi.PredefinedNeuronSet(node_set="Layer1")

stim_neuron_set = obi.PredefinedNeuronSet(node_set="Layer1", random_sample=[10, 20])
stim_times = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=3, interval=1000.0)  # in ms!!
current_stimulus = obi.ConstantCurrentClampSomaticStimulus(timestamps=stim_times, duration=5.0, neuron_set=stim_neuron_set, amplitude=[0.2, 0.5])
sync_current_stimulus = obi.ConstantCurrentClampSomaticStimulus(timestamps=stim_times, duration=100.0, neuron_set=stim_neuron_set, amplitude=0.1)

simulations_initialize = obi.SimulationsForm.Initialize(
    circuit=[circuit, circuit2],
    node_set=sim_neuron_set,
    simulation_length=100
)

"""
Fill form with Blocks
"""
simulations_form = obi.SimulationsForm(
                                    initialize=simulations_initialize,
                                    intracellular_location_sets={},
                                    extracellular_location_sets={},
                                    neuron_sets={"L1All": sim_neuron_set, "L1Sample": stim_neuron_set},
                                    synapse_sets={},
                                    timestamps={"StimTimes": stim_times},
                                    stimuli={"CurrentStimulus": current_stimulus, "SyncCurrentStimulus": sync_current_stimulus},
                                    recordings={},
                                    )

In [6]:
simulations_form.model_dump(mode="json")

{'type': 'SimulationsForm',
 'timestamps': {'StimTimes': {'type': 'RegularTimestamps',
   'start_time': 0.0,
   'sim_init_name': 'StimTimes',
   'number_of_repetitions': 3,
   'interval': 1000.0}},
 'stimuli': {'CurrentStimulus': {'type': 'ConstantCurrentClampSomaticStimulus',
   'timestamps': {'type': 'RegularTimestamps',
    'start_time': 0.0,
    'sim_init_name': 'StimTimes',
    'number_of_repetitions': 3,
    'interval': 1000.0},
   'sim_init_name': 'CurrentStimulus',
   'delay': 0.0,
   'duration': 5.0,
   'neuron_set': {'type': 'PredefinedNeuronSet',
    'sim_init_name': 'L1Sample',
    'random_sample': [10, 20],
    'random_seed': 0,
    'node_set': 'Layer1'},
   'represents_physical_electrode': False,
   'amplitude': [0.2, 0.5]},
  'SyncCurrentStimulus': {'type': 'ConstantCurrentClampSomaticStimulus',
   'timestamps': {'type': 'RegularTimestamps',
    'start_time': 0.0,
    'sim_init_name': 'StimTimes',
    'number_of_repetitions': 3,
    'interval': 1000.0},
   'sim_init_name

In [7]:
grid_scan = obi.GridScan(form=simulations_form, output_root='../../obi-output/circuit_simulations/grid_scan')
grid_scan.multiple_value_parameters(display=True)
grid_scan.coordinate_parameters(display=True)
# grid_scan.coordinate_instances(display=True)
grid_scan.execute(processing_method='generate')


MULTIPLE VALUE PARAMETERS
stimuli.CurrentStimulus.amplitude: [0.2, 0.5]
neuron_sets.L1Sample.random_sample: [10, 20]
initialize.circuit: [Circuit(type='Circuit', name='ToyCircuit-S1-6k', path='/Users/pokorny/Data/Circuits/ToyCircuit-S1-6k/circuit_config.json'), Circuit(type='Circuit', name='nbS1-HEX0-beta', path='/Users/pokorny/Data/Circuits/ToyCircuit-S1-6k/circuit_config.json')]

COORDINATE PARAMETERS
stimuli.CurrentStimulus.amplitude: 0.2, neuron_sets.L1Sample.random_sample: 10, initialize.circuit: ToyCircuit-S1-6k
stimuli.CurrentStimulus.amplitude: 0.2, neuron_sets.L1Sample.random_sample: 10, initialize.circuit: nbS1-HEX0-beta
stimuli.CurrentStimulus.amplitude: 0.2, neuron_sets.L1Sample.random_sample: 20, initialize.circuit: ToyCircuit-S1-6k
stimuli.CurrentStimulus.amplitude: 0.2, neuron_sets.L1Sample.random_sample: 20, initialize.circuit: nbS1-HEX0-beta
stimuli.CurrentStimulus.amplitude: 0.5, neuron_sets.L1Sample.random_sample: 10, initialize.circuit: ToyCircuit-S1-6k
stimuli.Cur

{0: None, 1: None, 2: None, 3: None, 4: None, 5: None, 6: None, 7: None}

In [8]:
# Deserialization
# grid_scan_ds = obi.deserialize_obi_object_from_json_file("../../obi-output/circuit_simulations/grid_scan/run_scan_config.json")